In [5]:
import pandas as pd
import altair as alt

df = pd.read_csv("combined_nyc_park_crime_stats.csv")
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Create quarter label + order
df["quarter_str"] = "Q" + df["quarter"].astype(str) + " " + df["year"].astype(str)
df["quarter_order"] = df["year"].astype(str) + "Q" + df["quarter"].astype(str)

# Crime colss
crime_columns = [
    "murder",
    "rape",
    "robbery",
    "felony_assault",
    "burglary",
    "grand_larceny",
    "grand_larceny_of_motor_vehicle"
]

# melt
df_melted = df.melt(
    id_vars=["borough", "park", "quarter", "year", "quarter_str", "quarter_order"],
    value_vars=crime_columns,
    var_name="crime_type",
    value_name="crime_count"
)

# quarter and type of crime group by 
agg_line = df_melted.groupby(["quarter_str", "quarter_order", "crime_type"]).agg(
    crime_count=("crime_count", "sum")
).reset_index()

# sort
agg_line = agg_line.sort_values("quarter_order")

# radio
radio_binding = alt.binding_radio(
    options=[None,
        "murder",
        "rape",
        "robbery",
        "felony_assault",
        "burglary",
        "grand_larceny",
        "grand_larceny_of_motor_vehicle"
    ],
    labels=["All",
        "Murder",
        "Rape",
        "Robbery",
        "Felony Assault",
        "Burglary",
        "Grand Larceny",
        "Grand Larceny of Motor Vehicle"
    ],
    name="Select Crime Type: "
)



crime_filter = alt.selection_point(
    fields=["crime_type"],
    bind=radio_binding,
    empty=True
)

line_chart = alt.Chart(agg_line).mark_line(point=True).encode(
    x=alt.X("quarter_str:N", title="Quarter",
            sort=agg_line["quarter_str"].unique().tolist()),  # custom sort
    y=alt.Y("crime_count:Q", title="Crime Count"),
    color=alt.Color("crime_type:N", title="Crime Type"),
    tooltip=["quarter_str", "crime_type", "crime_count"]
).add_params(
    crime_filter
).transform_filter(
    crime_filter
).properties(
    title="Chronological Crime Trends Per Quarter in New York City Parks by Type in 2023 & 2024",
    width=1150,
    height=600
)

line_chart.save("interactive_crime_trends_chart.html")
line_chart.display()


/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/c

alt.Chart(...)